In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from utils import log_wrapper_for_logged_data, DICT_RC_PARAM

In [ ]:
df = pd.read_csv("../generator/data/cplex/run_time_d0.8_UDG_8vCPU.csv")

In [ ]:
df["log10 Process TTS"] = np.log10(df["Process TTS"])
df["log2 Process TTS"] = np.log2(df["Process TTS"])
df["log Process TTS"] = np.log(df["Process TTS"])
df["N"] = np.round(df["L"] ** 2 * df["Density"]).astype(int)

In [ ]:
dg = df[df["N"] > 750].set_index("N")
dg.sort_values(by="log2 Process TTS", inplace=True)
x = []
y = []
for n in np.unique(dg.index):
    z = dg.loc[n][-10:]["log2 Process TTS"]
    x += list(z.index)
    y += list(z.values)
x = np.array(x)
y = np.array(y)

In [ ]:
lr = LinearRegression()
lr.fit(x.reshape(-1, 1), y)
print(lr.score(x.reshape(-1, 1), y))

### Evaluate the boxplot with the log10 data to properly evaluate the data, then change tickers to get a nice visualization

In [ ]:
%matplotlib inline
plt.rcParams.update(DICT_RC_PARAM)
fig, ax = plt.subplots(figsize=(15, 10))
positions = np.unique(df["N"])
df.boxplot("log10 Process TTS", by="N", ax=ax, positions=positions, widths=35)
ax.plot(
    np.unique(x),
    np.log(2) / np.log(10) * lr.predict(np.unique(x).reshape(-1, 1)),
    label=r"Top $1\%$ TTS = $O(2^{" + str(np.round(lr.coef_[0], 4)) + r"N})$",
)
plt.suptitle("")
ax.set_title("")
ax.set_ylabel("TTS (seconds)")
ax.set_xlabel("$N$")
ax.set_xscale("linear")
ax.legend()
log_wrapper_for_logged_data(ax, y_axis=True, base=10)
plt.savefig("Process_TTS_scaling_d08.png")
ax.set_title("Time to Solution on CPLEX for LxL Unit Disk Union-Jack on 1000 samples")
plt.show()

In [ ]:
def get_cplex_runtimes(d):
    df = pd.read_csv(f"../generator/data/cplex/run_time_d{d}_UDG_8vCPU.csv")
    df["log2 Process TTS"] = np.log2(df["Process TTS"].astype(float))
    df["log10 Process TTS"] = np.log10(df["Process TTS"].astype(float))
    df["N"] = np.round(df["L"] ** 2 * df["Density"]).astype(int)
    return df


def get_regression_Ns(d_list, skip=500, ax=None, top_val=10):
    d_list.sort(reverse=True)
    scalings = []
    for d in d_list:
        df = get_cplex_runtimes(d)
        lr = LinearRegression()
        df["log2 Process TTS"] = np.log2(df["Process TTS"])
        df_groupby = df[["N", "log2 Process TTS"]].groupby("N")["log2 Process TTS"]
        z_all = (
            df_groupby.nlargest(top_val)
            .reset_index()[["N", "log2 Process TTS"]]
            .set_index("N")
        )
        z_regression = z_all[z_all.index > skip]
        x = z_regression.index.values.reshape(-1, 1)
        y = z_regression.values
        lr.fit(x, y)
        print(
            f"{d}: score:{lr.score(x, y)}, coef:{lr.coef_}, intercept:{lr.intercept_}"
        )

        if ax is None:
            fig, ax = plt.subplots(figsize=(15, 10))

        ax.scatter(z_all.index.values, 2 ** (z_all.values))
        ax.set_yscale("log")
        ax.plot(
            x,
            2 ** (lr.predict(x)),
            label=r"$\rho="
            + str(d)
            + r"$, $O(2^{"
            + str(np.round(lr.coef_[0][0], 4))
            + r"N})$",
        )
        scalings.append(get_intervals(x, y))

    ax.set_xlabel("N")
    ax.set_ylabel("TTS (seconds)")
    ax.set_title(
        f"Cplex runtime for different densities\n (LxL UJ-lattice, 1000 samples each)"
    )
    ax.legend()
    return scalings


def get_top_xpercent(top_x_percent=10):
    return lambda x: np.sort(x)[-int(len(x) * top_x_percent / 100) :]

In [ ]:
def get_intervals(x, y):
    from scipy.stats import t, linregress, normaltest

    res = linregress(x.flatten(), y.flatten())
    tinv = lambda p, df: abs(t.ppf(p / 2, df))
    ts = tinv(0.05, len(x) - 2)

    def convert(x):
        return x

    # print(f"slope (95%): {res.slope:.6f} +/- {ts*res.stderr:.6f}")
    # 95% CI is not symetric
    print(
        f"slope (95%): {convert(res.slope):.6f}, {convert(res.slope+ts*res.stderr):.6f}, {convert(res.slope-ts*res.stderr):.6f}"
    )
    converted = convert(res.slope)
    return (
        converted,
        converted - convert(res.slope - ts * res.stderr),
        convert(res.slope + ts * res.stderr) - converted,
    )

In [ ]:
d = 0.75
df = get_cplex_runtimes(d)

In [ ]:
np.all(df["Density"] == d), np.all(df.groupby("L").count() == 1001)

In [ ]:
densities = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
skip = 750
top_val = int(0.01 * 1000)  # 1% of 1000 seeds
scalings = get_regression_Ns(densities, top_val=top_val, skip=skip)
# score of 0.6, 0.65 too low

In [ ]:
plt.rcParams.update(DICT_RC_PARAM)

y = [x[0] for x in scalings]
errors = np.array([x[1:] for x in scalings]).T
fig, ax = plt.subplots(figsize=(15, 10))
ax.grid(visible=True)
ax.errorbar(x=densities, y=y, yerr=errors, marker="o", linestyle="-", capsize=4)
ax.set_xlabel(r"Filling fraction ($\rho$)")
ax.set_ylabel("Exponent Coefficient ($a$)")
ax.set
# ax.set_title("Top 1\% CPLEX TTS = $O(2^{aN})$, with $95\%$ confidence interval\n (residual not following normal distribution)")
plt.savefig("Process_TTS_scaling_various_densities.png")